## Imports

In [2]:
from ipynb.fs.full.Smooth_Utils import *

## Math Utils

In [3]:
import numpy as np

def primesfrom3to(n):
    """ Returns a array of primes, 3 <= p < n """
    sieve = np.ones(n//2, dtype=bool)
    for i in range(3,int(n**0.5)+1,2):
        if sieve[i//2]:
            sieve[i*i//2::i] = False
    return 2*np.nonzero(sieve)[0][1::]+1

def primesupto(n):
    return np.concatenate(([2], primesfrom3to(n)))

def pi(x):
    return 1 + len(primesfrom3to(x))


def modInverse(A, M):
    A, M = np.copy(A), np.copy(M)
    # print(A, M)
    # print("^ above")
    m0 = np.copy(M)
    y = np.zeros(np.shape(A), dtype=int)
    x = np.ones(np.shape(A), dtype=int)
    q = np.zeros(np.shape(A), dtype=int)
    running = np.ones(np.shape(A), dtype=bool)

    while True:
        should_stop = (A <= 1)
        if should_stop.all():
            break
        running[should_stop] = 0
        Ar, Mr = A[running], M[running]
        # print(Ar, Mr)
        q[running] = Ar // Mr
        M[running], A[running] = Ar % Mr, Mr
        x[running], y[running] = y[running], x[running] - q[running] * y[running]
    return np.where(x < 0, x + m0, x)

## Long Calls

In [4]:
estimate_runtime = False

if estimate_runtime:

    special_q_lower_bound = 450_000_000 # according to paper
    special_q_upper_bound = 11_100_000_000 # according to paper
    num_special_q_values = pi(special_q_upper_bound) - pi(special_q_lower_bound) # we run on all primes within the range
    
    b_1 = 1_100_000_000 # according to paper
    b_2 = 200_000_000 # according to paper
    # assume that the number of prime/ideals will be roughly b_k, according to The Number Field Sieve paper
    prime_ideals = primesupto(b_1)
    primes = primesupto(b_2)

## Estimating Runtime

In [5]:
if estimate_runtime:
    I = pow(2, 16) # according to paper
    J = pow(2, 15) # according to paper
    NPU_freq = 1e9 # 1 GHz, as speculated in neuro sieve paper
    num_NPU = 1000 # parallel processors
    
    seconds_per_q = (I * J) / NPU_freq
    q_per_machine = num_special_q_values / num_NPU
    
    walltime_seconds = seconds_per_q * q_per_machine
    
    walltime_days = walltime_seconds / (60 * 60 * 24)
    
    print(f'estimated runtime: {walltime_seconds}s = {walltime_days} days for {num_NPU} machines running in parallel.')

In [6]:
if estimate_runtime:
    
    print(f'there will be roughly {len(primes)} primes and {len(prime_ideals)} ({len(primes) + len(prime_ideals)} nodes total).')
    
    print(f'there are only {pi(max(I, J))} primes per row, so the vast majority of primes will trigger less than once per row.')

## Delayed Synapse

In our theoretical architecture, all synapses take 1 tick to transfer. In Lava, this is only true when a neuron is connected to itself - otherwise, synapses are instant. This synapse process builds in the delay to mimic the theoretical model.

In [7]:
class BSSSynapseProcessDelayed(BSSSynapseProcess):
    def __init__(self, in_shape = (1,), out_shape=(1,), name = 'BSSSynapseProcessDelayed',
        synapse_weights = 0
    ):
        super().__init__(in_shape=in_shape, out_shape=out_shape, name=name, synapse_weights=synapse_weights)

        self.previous_values = Var(shape=in_shape, init=0)
        self.is_delayed = Var(shape=(1,), init=True)
        self.is_init = Var(shape=(1,), init=True)

    def connect(self, sending_neuron, out_port, receiving_neuron, in_port):
        out_port.connect(self.synapse_in)
        self.synapse_out.connect(in_port)
        self.is_delayed.init = (sending_neuron != receiving_neuron)

    def reset(self, init):
        # print('resetting delayed synapse')
        if not init:
            self.previous_values.set(np.zeros(np.shape(self.previous_values)))
            self.is_init.set(np.array([True]))
            # print(f'set init to {self.is_init.get()}')

@implements(proc=BSSSynapseProcessDelayed, protocol=LoihiProtocol)
@requires(CPU)
class BSSSynapseProcessDelayedModel(AbstractBSSSynapseProcessModel):
    previous_values: np.ndarray = LavaPyType(np.ndarray, float)
    is_delayed: bool = LavaPyType(bool, bool)
    is_init: bool = LavaPyType(bool, bool)

    def run_spk(self):
        if self.synapse_in.csp_ports and self.synapse_out.csp_ports:
            if self.is_init and not self.is_delayed:
                if self.synapse_in.probe():
                    # print('taking out the trash')
                    self.synapse_in.recv()
                in_results = np.zeros(np.shape(self.synapse_in))
            else:
                in_results = self.synapse_in.recv()
            # print(in_results, self.previous_values)
            # ^ if it's a self-loop, can't expect to receive anything at t = 0
            self.is_init = False
            # sending = (self.synapse_weights @ (self.previous_values if self.is_delayed else in_results))
            self.synapse_out.send((self.synapse_weights @ (self.previous_values if self.is_delayed else in_results)))
            # print(f'results = {in_results}')
            # print(sending)
            self.previous_values = in_results

In [8]:
# now we'll test to make sure it works, with two different neurons connected with a delay

proc_1 = SimpleLIFProcess(shape=(1,), physical_properties=[SimpleLIFProperties(0, 3, 1)], name='proc_1') # tonic every 3
proc_2 = SimpleLIFProcess(shape=(1,), physical_properties=[SimpleLIFProperties(0, 0, -1)], name='proc_2') # requires input to spike

synapse = BSSSynapseProcessDelayed(synapse_weights = 1)
synapse.connect(proc_1, proc_1.spike_out, proc_2, proc_2.synapse_in)

num_steps = 10
monitor_1 = Monitor()
monitor_1.probe(proc_1.spike_out, num_steps)
monitor_2 = Monitor()
monitor_2.probe(proc_2.spike_out, num_steps)

run_cfg = Loihi1SimCfg()
proc_1.run(condition=RunSteps(num_steps=num_steps), run_cfg=run_cfg)

data_1 = monitor_1.get_data()[proc_1.name]['spike_out']
data_2 = monitor_2.get_data()[proc_2.name]['spike_out']

proc_1.stop()

assert data_2[0][0] == 0
for i in range(num_steps - 1):
    assert data_2[i + 1][0] == data_1[i][0]

In [9]:
# now testing with one neuron connected to itself

proc_1 = SimpleLIFProcess(shape=(1,), physical_properties=[SimpleLIFProperties(0, 4, 1)], name='proc_1') # tonic every 4
synapse = BSSSynapseProcessDelayed(synapse_weights = 0.5)
synapse.connect(proc_1, proc_1.spike_out, proc_1, proc_1.synapse_in)

num_steps = 10
monitor_1_spikes = Monitor()
monitor_1_spikes.probe(proc_1.spike_out, num_steps)
monitor_1_voltage = Monitor()
monitor_1_voltage.probe(proc_1.V_m, num_steps)

run_cfg = Loihi1SimCfg()
proc_1.run(condition=RunSteps(num_steps=num_steps), run_cfg=run_cfg)

spike_data = monitor_1_spikes.get_data()[proc_1.name]['spike_out']
voltage_data = monitor_1_voltage.get_data()[proc_1.name]['V_m']

proc_1.stop()

print(spike_data)
print(voltage_data)

[[0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]]
[[1. ]
 [2. ]
 [3. ]
 [4. ]
 [1.5]
 [2.5]
 [3.5]
 [4.5]
 [1.5]
 [2.5]]


## Integer Precision Synapses

Some of the synapses involved in this structure (the ones that regulate the voltage of the prime neurons) need to have integer precision, so those synapse processes/models are defined below.

In [10]:
class BSSSynapseProcessIntPrecision(BSSSynapseProcess):
    def __init__(self, in_shape = (1,), out_shape=(1,), name = 'BSSSynapseProcessIntPrecision',
        synapse_weights = 0
    ):
        super().__init__(in_shape=in_shape, out_shape=out_shape, name=name, synapse_weights=synapse_weights)
    
class AbstractBSSSynapseProcessModelIntPrecision(PyLoihiProcessModel):
    synapse_weights: np.ndarray = LavaPyType(np.ndarray, int) # A weight matrix for synapses, used to connect this layer of neurons
                                                                # to another layer of the same shape (or it can feed into itself, but this would be rare)
    synapse_in: PyInPort = LavaPyType(PyInPort.VEC_DENSE, bool, precision=1)
    synapse_out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, int)

    def run_spk(self):
        if self.synapse_in.csp_ports and self.synapse_out.csp_ports:
            in_results = self.synapse_in.recv()
            self.synapse_out.send(self.synapse_weights @ in_results)

@implements(proc=BSSSynapseProcessIntPrecision, protocol=LoihiProtocol)
@requires(CPU)
class BSSSynapseProcessModelIntPrecision(AbstractBSSSynapseProcessModelIntPrecision):
    pass

class BSSSynapseProcessDelayedIntPrecision(BSSSynapseProcessDelayed):
    def __init__(self, in_shape = (1,), out_shape=(1,), name = 'BSSSynapseProcessDelayedIntPrecision',
        synapse_weights = 0
    ):
        super().__init__(in_shape=in_shape, out_shape=out_shape, name=name, synapse_weights=synapse_weights)

    def reset(self, init):
        # print('resetting int precision')
        super().reset(init)
        # print('done int')
        
@implements(proc=BSSSynapseProcessDelayedIntPrecision, protocol=LoihiProtocol)
@requires(CPU)
class BSSSynapseProcessDelayedModelIntPrecision(AbstractBSSSynapseProcessModelIntPrecision):
    previous_values: np.ndarray = LavaPyType(np.ndarray, int)
    is_delayed: bool = LavaPyType(bool, bool)
    is_init: bool = LavaPyType(bool, bool)

    def run_spk(self):
        if self.synapse_in.csp_ports and self.synapse_out.csp_ports:
            if self.is_init and not self.is_delayed:
                if self.synapse_in.probe():
                    # print('taking out the trash')
                    self.synapse_in.recv()
                in_results = np.zeros(np.shape(self.synapse_in))
            else:
                in_results = self.synapse_in.recv()
            # print(in_results, self.previous_values)
            # ^ if it's a self-loop, can't expect to receive anything at t = 0
            self.is_init = False
            # sending = (self.synapse_weights @ (self.previous_values if self.is_delayed else in_results))
            self.synapse_out.send((self.synapse_weights @ (self.previous_values if self.is_delayed else in_results)))
            # print(f'results = {in_results}')
            # print(sending)
            self.previous_values = in_results

## Simple Lattice, Single Layer

This experiment tests a simple lattice sieve with only one prime number (100, even though it's not actually prime). It uses one neuron with a 4x4 synapse ( (n + 3) x (n + 3) for n primes ).

In [11]:
p = 100 # in reality this would be a prime
C = 4
D = 6
f = 52 # in reality this would be -u_1 * (inverse of u_2, mod p), all mod p
g = -(f + D + 1) % p
V_0 = (C * f)%p
B = 2

main_layer = SimpleLIFProcess(shape=(4,), physical_properties=[SimpleLIFProperties(1, D + 1, 1),
                                                               SimpleLIFProperties(0, 0, -1),
                                                               SimpleLIFProperties(V_0, p, 1, False),
                                                               SimpleLIFProperties(0, 0, -B)], name='main')
# index 0 = row changer, index 1 = relay, index 2 = prime, index 3 = bottom node
synapse_weights = [
    [0, 0, 0, 0],
    [1, 0, 0, 0],
    [g, 0, -p, 0],
    [0, -np.log(p), np.log(p), 0]
]

synapse = BSSSynapseProcessDelayed(in_shape=(4,), out_shape=(4,), synapse_weights = synapse_weights, name='main_synapse')
synapse.connect(main_layer, main_layer.spike_out, main_layer, main_layer.synapse_in)

num_steps = (D + 1) * (2 * C + 1) # we need to include d = D + 1 at the end in case (C, D) caused prime to fire, giving the bottom layer enough time to receive
spike_monitor = Monitor()
spike_monitor.probe(main_layer.spike_out, num_steps)
voltage_monitor = Monitor()
voltage_monitor.probe(main_layer.V_m, num_steps)

run_cfg = Loihi1SimCfg()
main_layer.run(condition=RunSteps(num_steps=num_steps), run_cfg=run_cfg)

spike_data = spike_monitor.get_data()[main_layer.name]['spike_out']
voltage_data = voltage_monitor.get_data()[main_layer.name]['V_m']

main_layer.stop()

# print(len(voltage_data))
# print(voltage_data)
# print(spike_data)

spike_times = np.where(spike_data[:, 3])[0]
for time in spike_times:
    c = (time - 1)//(D + 1) - C
    d = (time) % (D + 1)
    print(f'spiked at t={time}, c={c}, d={d}')
    

spiked at t=46, c=2, d=4


## Simple Lattice, Multiple Layers

Now we split into three layers, causing the total number of matrix elements to drop to n^2 + 2n. There are only 3n pieces of information, though, since the primes to primes matrix is diagonal. In the future we could consider adding a special synapse process that specifically assumes the matrix is diagonal and stores n entries instead of n^2.

In [12]:
p = 100 # in reality this would be a prime
C = 4
D = 6
f = 52 # in reality this would be -u_1 * (inverse of u_2, mod p), all mod p
g = -(f + D + 1) % p
V_0 = (C * f)%p
B = 2

top_layer = SimpleLIFProcess(shape=(1,), physical_properties=[SimpleLIFProperties(1, D + 1, 1)], name='top')
primes_layer = SimpleLIFProcess(shape=(2,), physical_properties=[SimpleLIFProperties(0, 0, -1),
                                                               SimpleLIFProperties(V_0, p, 1, False)], name='primes')
bottom_layer = SimpleLIFProcess(shape=(1,), physical_properties=[SimpleLIFProperties(0, 0, -B)], name='bottom')

# index 0 = relay, index 1 = prime

top_to_primes_weights = [
    [1],
    [g]
]

primes_to_primes_weights = [
    [0, 0],
    [0, -p]
]

primes_to_bottom_weights = [
    [-np.log(p), np.log(p)]
]

top_to_primes = BSSSynapseProcessDelayed(in_shape=(1,), out_shape=(2,), synapse_weights = top_to_primes_weights, name='top_to_primes')
primes_to_primes = BSSSynapseProcessDelayed(in_shape=(2,), out_shape=(2,), synapse_weights = primes_to_primes_weights, name='primes_to_primes')
primes_to_bottom = BSSSynapseProcessDelayed(in_shape=(2,), out_shape=(1,), synapse_weights = primes_to_bottom_weights, name='primes_to_bottom')

top_to_primes.connect(top_layer, top_layer.spike_out, primes_layer, primes_layer.synapse_in)
primes_to_primes.connect(primes_layer, primes_layer.spike_out, primes_layer, primes_layer.synapse_in)
primes_to_bottom.connect(primes_layer, primes_layer.spike_out, bottom_layer, bottom_layer.synapse_in)

num_steps = (D + 1) * (2 * C + 1) # we need to include d = D + 1 at the end in case (C, D) caused prime to fire, giving the bottom layer enough time to receive
spike_monitor = Monitor()
spike_monitor.probe(bottom_layer.spike_out, num_steps)
voltage_monitor = Monitor()
voltage_monitor.probe(primes_layer.V_m, num_steps)

run_cfg = Loihi1SimCfg()
top_layer.run(condition=RunSteps(num_steps=num_steps), run_cfg=run_cfg)

spike_data = spike_monitor.get_data()[bottom_layer.name]['spike_out']
voltage_data = voltage_monitor.get_data()[primes_layer.name]['V_m']

top_layer.stop()

# print(len(voltage_data))
# print(voltage_data)
# print(spike_data)

spike_times = np.where(spike_data[:, 0])[0]
for time in spike_times:
    c = (time - 1)//(D + 1) - C
    d = (time) % (D + 1)
    print(f'spiked at t={time}, c={c}, d={d}')
    

spiked at t=46, c=2, d=4


## Simple Monitor
Copied from Smooth_GNFS.

In [13]:
class SimpleMonitor(AbstractProcess):
    def __init__(self, num_steps, shape=(1,)):
        super().__init__()
        self.in_port = InPort(shape=shape)
        self.cache = Var(shape=(num_steps,)+shape, init=0)
        self.step_count = Var(shape=(1,), init=np.array([0], dtype=int))

    def reset(self):
        self.cache.set(np.zeros(np.shape(self.cache), dtype=bool))
        self.step_count.set(np.array([0], dtype=int))

@implements(proc=SimpleMonitor, protocol=LoihiProtocol)
@requires(CPU)
class SimpleMonitorModel(PyLoihiProcessModel):
    in_port: PyInPort = LavaPyType(PyInPort.VEC_DENSE, bool, precision=1)
    cache: np.ndarray = LavaPyType(np.ndarray, bool, precision=1)
    step_count: int = LavaPyType(int, int)

    def run_spk(self):
        if self.step_count >= len(self.cache):
            return
        in_recv = self.in_port.recv()
        self.cache[self.step_count] = in_recv
        self.step_count += 1

## Utils

In [14]:
class Utils:
    def __init__(self, I, J):
        self.I = I
        self.J = J

    def i_from_spike_time(self, t):
        return (t - 2)//self.J - self.I//2
    
    def j_from_spike_time(self, t):
        return (t - 1) % self.J
    
    def t_from_i_j(self, i, j):
        return (i + self.I//2) * self.J + j

## Coprime Siever Class

In [37]:
class CoprimeSiever:
    def __init__(self, I, J, top_layer, bottom_layer):
        self.I = I
        self.J = J
        self.generate_neurons(top_layer, bottom_layer)
    def generate_neurons(self, top_layer, bottom_layer):
        self.primes_using = primesupto(min(self.I//2+1, self.J))
        num_p = len(self.primes_using)
        self.i_primes_layer = SimpleLIFProcessIntPrecision(shape=(num_p,), physical_properties=[SimpleLIFProperties(0, 0, 0) for _ in range(num_p)], name='i_primes_gcd')
        self.j_primes_layer = SimpleLIFProcessIntPrecision(shape=(num_p,), physical_properties=[SimpleLIFProperties(0, 0, 0) for _ in range(num_p)], name='j_primes_gcd')
        self.bottom_layer = SimpleLIFProcess(shape=(1,), physical_properties=[SimpleLIFProperties(0, 0, -1)], name='bottom_gcd')
        self.izero_layer = SimpleLIFProcessIntPrecision(shape=(2,), physical_properties=[SimpleLIFProperties(0, 0, 0) for _ in range(2)], name='izero_gcd')
        # ^ left izero neuron fires when I reaches 0 and then resets, right neuron latches
        top_to_j_weights = [
            [-(self.J + p)] for p in self.primes_using
        ]

        i_to_j_weights = [
            [self.J if a == b else 0 for a in range(num_p)]
            for b in range(num_p)
        ]

        j_to_j_weights = [
            [-self.primes_using[a] if a == b else 0 for a in range(num_p)]
            for b in range(num_p)
        ]
        
        self.top_to_i_primes = BSSSynapseProcessDelayedIntPrecision(in_shape=(1,), out_shape=(num_p,), synapse_weights=np.ones((num_p, 1)), name='top_to_i_primes_gcd')
        self.top_to_j_primes = BSSSynapseProcessDelayedIntPrecision(in_shape=(1,), out_shape=(num_p,), synapse_weights=top_to_j_weights, name='top_to_j_primes_gcd')
        self.top_to_izero = BSSSynapseProcessDelayedIntPrecision(in_shape=(1,), out_shape=(2,), synapse_weights=np.array([[1], [-1]]), name='top_to_izero_gcd')
        self.i_primes_to_j_primes = BSSSynapseProcessDelayedIntPrecision(in_shape=(num_p,), out_shape=(num_p,), synapse_weights=i_to_j_weights, name='i_to_j_gcd')
        self.j_primes_to_j_primes = BSSSynapseProcessDelayedIntPrecision(in_shape=(num_p,), out_shape=(num_p,), synapse_weights=j_to_j_weights, name='j_to_j_gcd')
        self.j_to_bottom = BSSSynapseProcessDelayedIntPrecision(in_shape=(num_p,), out_shape=(1,), synapse_weights=np.ones((1, num_p)), name='j_to_bottom_gcd')
        self.izero_to_izero = BSSSynapseProcessDelayedIntPrecision(in_shape=(2,), out_shape=(2,), synapse_weights=np.array([[0, 0], [1, 0]]), name='izero_to_izero_gcd')
        self.izero_to_bottom = BSSSynapseProcessDelayedIntPrecision(in_shape=(2,), out_shape=(1,), synapse_weights=np.array([[0, 1]]), name='izero_to_bottom_gcd')
        self.bottom_to_bottom = BSSSynapseProcessDelayedIntPrecision(in_shape=(1,), out_shape=(1,), synapse_weights=-1, name='coprimes_to_bottom')
        
        self.top_to_i_primes.connect(top_layer, top_layer.spike_out, self.i_primes_layer, self.i_primes_layer.synapse_in)
        self.top_to_j_primes.connect(top_layer, top_layer.spike_out, self.j_primes_layer, self.j_primes_layer.synapse_in)
        self.top_to_izero.connect(top_layer, top_layer.spike_out, self.izero_layer, self.izero_layer.synapse_in)
        self.i_primes_to_j_primes.connect(self.i_primes_layer, self.i_primes_layer.spike_out, self.j_primes_layer, self.j_primes_layer.synapse_in)
        self.j_primes_to_j_primes.connect(self.j_primes_layer, self.j_primes_layer.spike_out, self.j_primes_layer, self.j_primes_layer.synapse_in)
        self.j_to_bottom.connect(self.j_primes_layer, self.j_primes_layer.spike_out, self.bottom_layer, self.bottom_layer.synapse_in)
        self.izero_to_izero.connect(self.izero_layer, self.izero_layer.spike_out, self.izero_layer, self.izero_layer.synapse_in)
        self.izero_to_bottom.connect(self.izero_layer, self.izero_layer.spike_out, self.bottom_layer, self.bottom_layer.synapse_in)
        self.bottom_to_bottom.connect(self.bottom_layer, self.bottom_layer.spike_out, bottom_layer, bottom_layer.synapse_in)
    def update_neurons(self, init):
        i_phys = [SimpleLIFProperties((-self.I//2)%p, p, 0) for p in self.primes_using]
        j_phys = [SimpleLIFProperties(
            self.J if ((-self.I//2)%p == 0) else 0, self.J + p, 1, False) for p in self.primes_using]
        self.i_primes_layer.reset(physical_properties=i_phys, init=init)
        self.j_primes_layer.reset(physical_properties=j_phys, init=init)
        self.izero_layer.reset(physical_properties=[
            SimpleLIFProperties(0, self.I//2, 0),
            SimpleLIFProperties(0, 1, 0, False) # latches output from the other neuron
        ], init=init)
        # for phys in i_phys, j_phys:
        #     for prop in phys:
        #         for name, val in (
        #             (prop.V_0, 'v_0'),
        #             (prop.V_leak, 'thresh'),
        #             (prop.leak_amt, 'leak')):
        #             print(name, val)

        for synapse in self.top_to_i_primes, self.top_to_j_primes, self.i_primes_to_j_primes, self.j_primes_to_j_primes, self.j_to_bottom, self.izero_to_bottom:
            synapse.reset(init)
    def sieve_once_through(self, bottom_layer, init):
        num_steps = self.I * self.J
        run_cfg = Loihi1SimCfg()
        monitor = SimpleMonitor(num_steps)
        bottom_layer.spike_out.connect(monitor.in_port)
        self.update_neurons(init=init)
        bottom_layer.run(condition=RunSteps(num_steps=num_steps), run_cfg=run_cfg)
        # for t in range(num_steps):
        #     top_layer.run(condition=RunSteps(num_steps=1), run_cfg=run_cfg)
        #     print(self.j_primes_layer.V_m.get())
        spike_times = np.where(monitor.cache.get())[0]
        monitor.reset()
        top_layer.stop()
        return spike_times

In [38]:
# testing the coprime siever
J = 10
top_layer = SimpleLIFProcessIntPrecision(shape=(1,), physical_properties=[SimpleLIFProperties(1, J, 1)], name='top')
bottom_layer = SimpleLIFProcessIntPrecision(shape=(1,), physical_properties=[SimpleLIFProperties(0, 1, 1)], name='bottom')
coprime_siever = CoprimeSiever(I=6, J=J, top_layer=top_layer, bottom_layer=bottom_layer)

print(coprime_siever.primes_using)
# ^ usually this will be done by an outer siever

results = coprime_siever.sieve_once_through(bottom_layer, init=True)
utils = Utils(coprime_siever.I, coprime_siever.J)
for t in results:
    print(f't={t}, i={utils.i_from_spike_time(t)}, j={utils.j_from_spike_time(t)}')

[2 3]
t=0, i=-4, j=9
t=1, i=-4, j=0
t=2, i=-3, j=1
t=3, i=-3, j=2
t=5, i=-3, j=4
t=6, i=-3, j=5
t=8, i=-3, j=7
t=9, i=-3, j=8
t=11, i=-3, j=0
t=12, i=-2, j=1
t=14, i=-2, j=3
t=16, i=-2, j=5
t=18, i=-2, j=7
t=20, i=-2, j=9
t=21, i=-2, j=0
t=22, i=-1, j=1
t=23, i=-1, j=2
t=24, i=-1, j=3
t=25, i=-1, j=4
t=26, i=-1, j=5
t=27, i=-1, j=6
t=28, i=-1, j=7
t=29, i=-1, j=8
t=30, i=-1, j=9
t=31, i=-1, j=0
t=41, i=0, j=0
t=42, i=1, j=1
t=43, i=1, j=2
t=44, i=1, j=3
t=45, i=1, j=4
t=46, i=1, j=5
t=47, i=1, j=6
t=48, i=1, j=7
t=49, i=1, j=8
t=50, i=1, j=9
t=51, i=1, j=0
t=52, i=2, j=1
t=54, i=2, j=3
t=56, i=2, j=5
t=58, i=2, j=7


## Lattice Siever Class

In [39]:
from sympy import poly
from sympy.abc import x as var_x
from sympy.polys import polytools
import time
def set_variable(variable, value, init):
    if init:
        variable.init = np.array(value)
    else:
        variable.set(np.array(value))

class LatticeSiever:
    def __init__(
        self, name, b_0, b_1, B, I, J, top_layer, bottom_layer
    ):
        self.name = name
        self.B = B
        self.I = I
        self.J = J
        rp_pairs = self.get_rp_pairs(b_0, b_1) # virtual function
        self.r = np.array([r for (r, p) in rp_pairs])
        self.p = np.array([p for (r, p) in rp_pairs])
        self.generate_neurons(name, top_layer, bottom_layer)

    def get_rp_pairs(self, b_0, b_1): # virtual
        return ()

    def get_norm(self, a, b): # virtual
        return 0

    def check_any_factors(self, a, b): 
        for r, p in zip(self.r, self.p):
            if (a - b * r) % p == 0:
                return True
        return False

    def get_reduced_norm(self, a, b):
        norm = self.get_norm(a, b)
        for r, p in zip(self.r, self.p):
            if (a - b * r) % p == 0:
                while norm % p == 0:
                    norm //= p
        return norm

    def check_smooth(self, a, b): 
        return abs(self.get_reduced_norm(a, b)) == 1

    def generate_neurons(self, name, top_layer, bottom_layer):
        num_p = len(self.p)

        primes_to_primes_weights = [
            [-self.p[i] if i == j else 0 for j in range(num_p)]
            for i in range(num_p)
        ]
        
        primes_to_bottom_weights = [
            [np.log(self.p[i]) for i in range(num_p)]
        ]
        
        self.primes_layer = SimpleLIFProcessIntPrecision(shape=(num_p,), physical_properties=[SimpleLIFProperties(0, 0, 0) for _ in range(num_p)], name=f'primes_{name}')
        self.bottom_layer = SimpleLIFProcess(shape=(1,), physical_properties=[SimpleLIFProperties(0, 0, -self.B)], name=f'bottom_{name}')
        self.relay = SimpleLIFProcessIntPrecision(shape=(1,), physical_properties=[SimpleLIFProperties(0, 0, -1)], name=f'relay_{name}')
        
        self.top_to_primes = BSSSynapseProcessDelayedIntPrecision(in_shape=(1,), out_shape=(num_p,), synapse_weights = 0, name=f'top_to_primes_{name}')
        self.primes_to_primes = BSSSynapseProcessDelayedIntPrecision(in_shape=(num_p,), out_shape=(num_p,), synapse_weights = primes_to_primes_weights, name=f'primes_to_primes_{name}')
        self.primes_to_bottom = BSSSynapseProcessDelayed(in_shape=(num_p,), out_shape=(1,), synapse_weights = primes_to_bottom_weights, name=f'primes_to_bottom_{name}')
        self.top_to_relay = BSSSynapseProcessDelayedIntPrecision(in_shape=(1,), out_shape=(1,), synapse_weights=1, name=f'top_to_relay_{name}')
        self.relay_to_bottom = BSSSynapseProcessDelayed(in_shape=(1,), out_shape=(1,), synapse_weights=-sum(primes_to_bottom_weights[0]), name=f'relay_to_bottom_{name}')
        self.bottom_to_bottom = BSSSynapseProcessDelayedIntPrecision(in_shape=(1,), out_shape=(1,), synapse_weights=1, name=f'{name}_to_bottom')
        
        self.top_to_primes.connect(top_layer, top_layer.spike_out, self.primes_layer, self.primes_layer.synapse_in)
        self.primes_to_primes.connect(self.primes_layer, self.primes_layer.spike_out, self.primes_layer, self.primes_layer.synapse_in)
        self.primes_to_bottom.connect(self.primes_layer, self.primes_layer.spike_out, self.bottom_layer, self.bottom_layer.synapse_in)
        self.top_to_relay.connect(top_layer, top_layer.spike_out, self.relay, self.relay.synapse_in)
        self.relay_to_bottom.connect(self.relay, self.relay.spike_out, self.bottom_layer, self.bottom_layer.synapse_in)
        self.bottom_to_bottom.connect(self.bottom_layer, self.bottom_layer.spike_out, bottom_layer, bottom_layer.synapse_in)

    def update_neurons(self, u_1, u_2, v_1, v_2, init, debug=False, verbose=False):
        # every q, we will need to update the primes layer and the top to prime synapse since these are dependent on q
        mod_inp = (v_1 - v_2 * self.r) % self.p
        which_special_cases = (mod_inp == 0)
        mod_inp[which_special_cases] = 1 # ignore these cases since the inverse is unsolvable
        if verbose:
            print(mod_inp, self.p)
            print('^ inputs to inv')
        
        modded = modInverse(mod_inp, self.p)
        if verbose:
            print(modded)
        ti = (u_1 - u_2 * self.r)
        if verbose:
            print(f'u={ti}')
            print(f'f={-(modded * ti)}')
        f = -(modded * ti) % self.p
        g_inner = -(f + self.J) % self.p
        g_inner[which_special_cases] = 0
        g = np.array([g_inner])
        V_0 = (self.I//2 * f)%self.p
        if verbose:
            print(f'V_0={V_0}, p={self.p}, special cases={which_special_cases}')
            print(f'g={g}')
        props = [SimpleLIFProperties(
            V_0[i], p, (0 if which_special_cases[i] else 1), False
        ) for i, p in enumerate(self.p)]
        self.primes_layer.reset(physical_properties=props, init=init, typ=int)
        set_variable(self.top_to_primes.synapse_weights, g.T, init)
        
        # print('variable values ----')
        # print('primes vm', self.primes_layer.V_m.get())
        # print('primes leak', self.primes_layer.leak_amt.get())
        # print('primes threshold', self.primes_layer.V_leak.get())
        # print('g', self.top_to_primes.synapse_weights.get())
        # print('----')
        for synapse in self.top_to_primes, self.primes_to_primes, self.primes_to_bottom, self.top_to_relay, self.relay_to_bottom, self.bottom_to_bottom:
            synapse.reset(init)

## Primes Siever Class

In [40]:
class PrimesSiever(LatticeSiever):
    def __init__(
        self, name, b_0, b_1, B, I, J, top_layer, bottom_layer, m
    ):
        self.m = m
        super().__init__(name, b_0, b_1, B, I, J, top_layer, bottom_layer)

    def get_rp_pairs(self, b_0, b_1):
        all_primes = primesupto(b_1)
        return [(self.m, p) for p in all_primes if p >= b_0]

    def get_norm(self, a, b):
        return round(a - b * self.m)

## Prime Ideals Siever Class

In [41]:
class PrimeIdealsSiever(LatticeSiever):
    def __init__(
        self, name, b_0, b_1, B, I, J, top_layer, bottom_layer, coefficients
    ):
        self.coefficients = coefficients
        self.d = len(coefficients) - 1
        self.sympy_poly = 0
        for i, coeff in enumerate(self.coefficients):
            self.sympy_poly += (coeff * var_x ** i)
        super().__init__(name, b_0, b_1, B, I, J, top_layer, bottom_layer)

    def get_rp_pairs(self, b_0, b_1):
        all_primes = primesupto(b_1)
        return [(r, p) for p in all_primes for r in self.find_roots(p) if p >= b_0]

    def find_roots(self, prime):
        factors = polytools.factor_list(polytools.factor(self.sympy_poly, modulus=prime))[1]
        factor_nums = []
        for factor_p, one in factors:
            if one != 1:
                continue
            if polytools.degree(factor_p) != 1:
                continue
            factor_nums.append(-poly(factor_p).TC() % prime)
        return factor_nums

    def polynomial_f(self, x):
        return sum([coeff * (x**n) for n, coeff in enumerate(self.coefficients)])

    def get_norm(self, a, b):
        return round(pow(b, self.d) * self.polynomial_f(a/b))


## GNFSiever Class

In [42]:
class GNFSiever:
    def __init__(self, 
                 coefficients, m, skew,
                 b_0_primes, b_1_primes, B_primes,
                 b_0_ideals, b_1_ideals, B_ideals,
                 I, J):
        self.top_layer = SimpleLIFProcessIntPrecision(shape=(1,), physical_properties=[SimpleLIFProperties(1, coprime_siever.J, 1)], name='top')
        self.bottom_layer = SimpleLIFProcessIntPrecision(shape=(1,), physical_properties=[SimpleLIFProperties(0, 0, -2)], name='bottom')
        self.coprime_siever = CoprimeSiever(I, J, top_layer=self.top_layer, bottom_layer=self.bottom_layer)
        self.prime_siever = PrimesSiever('primes', b_0_primes, b_1_primes, B_primes, I, J, self.top_layer, self.bottom_layer, m)
        self.prime_ideals_siever = PrimeIdealsSiever('ideals', b_0_ideals, b_1_ideals, B_ideals, I, J, self.top_layer, self.bottom_layer, coefficients)
        self.I = I
        self.J = J
        self.skew = skew
        self.init = True

    def find_basis(self, q, s):
        u = np.array([q, 0], dtype=float)
        v = np.array([s, self.skew], dtype=float)
        v_norm = np.linalg.norm(v)
        if v_norm > np.linalg.norm(u):
            temp = u
            u = v
            v = temp
            v_norm = q
        while v_norm <= np.linalg.norm(u):
            mult = round(np.dot(u, v) / (v_norm * v_norm))
            r = u - mult * v
            u = np.copy(v)
            v = r
            v_norm = np.linalg.norm(v)
        return (int(u[0]), int(u[1]/self.skew)), (int(v[0]), int(v[1]/self.skew))

    def update_neurons(self, u_1, u_2, v_1, v_2, debug, verbose):
        self.top_layer.reset(physical_properties=[SimpleLIFProperties(1, self.J, 1)], init=self.init, typ=int) # need to reset here just to update voltage
        self.coprime_siever.update_neurons(init=self.init)
        self.prime_siever.update_neurons(u_1, u_2, v_1, v_2, init=self.init, debug=debug, verbose=verbose)
        self.prime_ideals_siever.update_neurons(u_1, u_2, v_1, v_2, init=self.init, debug=debug, verbose=verbose)
        
    def sieve(self, q_min, q_max, debug=False, verbose=False):
        print('Starting sieve, finding all q values...')
        all_qs = primesupto(q_max)
        chosen_qs = all_qs[all_qs >= q_min]
        print('done.')
        num_steps_per_q = self.I * self.J
        run_cfg = Loihi1SimCfg()
        monitor = SimpleMonitor(num_steps_per_q)
        self.bottom_layer.spike_out.connect(monitor.in_port)
        output = []
        for q in chosen_qs:
            for s in self.prime_ideals_siever.find_roots(q):
                # time.sleep(2)
                if debug:
                    print(f'Now trying q={q}, s={s}')
                (u_1, u_2), (v_1, v_2) = self.find_basis(q, s)
                self.update_neurons(u_1, u_2, v_1, v_2, debug, verbose)
                self.top_layer.run(condition=RunSteps(num_steps=num_steps_per_q), run_cfg=run_cfg)
                if debug:
                    print('done')
                self.init = False
                spike_times = np.where(monitor.cache.get())[0]
                if verbose:
                    print(spike_times)
                output.append((q, s, (u_1, u_2), (v_1, v_2), spike_times))
                monitor.reset()
        self.top_layer.stop()
        print('Completed sieve.')
        return output

## Testing

In [51]:
I = 8
J = 7
utils = Utils(I=I, J=J)
siever = GNFSiever(
    coefficients=(8, 29, 15, 1), m=5, skew=1,
    b_0_primes=1, b_1_primes=15, B_primes=0.1,
    b_0_ideals=1, b_1_ideals=15, B_ideals=0,
    I=I, J=J) 
results = siever.sieve(30, 100)

Starting sieve, finding all q values...
done.
Completed sieve.


In [52]:
import primefac
def check_results(results):
    smooth_ab = []
    for q, s, (u_1, u_2), (v_1, v_2), spike_times in results:
        for t in spike_times:
            i = utils.i_from_spike_time(t)
            j = utils.j_from_spike_time(t)
            a = i * u_1 + j * v_1
            b = i * u_2 + j * v_2
            assert (u_1 - s * u_2)%q == 0, f'u = {(u_1, u_2)} was invalid for q {q}, s {s}!'
            assert (v_1 - s * v_2)%q == 0, f'v = {(v_1, v_2)} was invalid for q {q}, s {s}!'
            
            # initial_norm = siever.prime_ideals_siever.get_norm(a, b)
            # assert initial_norm % q == 0, f'norm {initial_norm} was not divisible by special q {q}!'
            # ab_gcd = abs(gcd(a, b))
            # without_q = False
            # if ab_gcd == q:
            #     a //= q
            #     b //= q
            #     ab_gcd = 1
            #     without_q = True
            ab_gcd=1
            print(f'q:{q}, s:{s}, t:{t} (fired at {t - 2}), i:{i}, j:{j}, a:{a}, b:{b} (u={(u_1, u_2)}, v={(v_1, v_2)}, ab-gcd={ab_gcd})')
            # assert ab_gcd == 1, f'{a} and {b} were not coprime! (gcd={ab_gcd})'

            # is_fully_smooth = True
            # for inner_sieve in siever.prime_siever, siever.prime_ideals_siever:
            #     if not inner_sieve.check_any_factors(a, b):
            #         print(f'Found an outlier on {inner_sieve.name} siever! Norm = {inner_sieve.get_norm(a, b)}')
            #         return None
            #     norm = inner_sieve.get_norm(a, b)
            #     reduced_norm = inner_sieve.get_reduced_norm(a, b)
            #     if inner_sieve == siever.prime_ideals_siever and not without_q:
            #         reduced_norm //= q
            #     is_smooth = abs(reduced_norm) == 1
            #     is_fully_smooth = is_fully_smooth and is_smooth
            #     print(f'{inner_sieve.name} norm {norm} reduced to {reduced_norm}, is smooth = {is_smooth}.')
            # if is_fully_smooth:
            #     smooth_ab.append((a, b))
    return smooth_ab
smooth_pairs = check_results(results)
print()
print('Smooth pairs:')
print(smooth_pairs)

q:31, s:18, t:6 (fired at 4), i:-4, j:5, a:35, b:-17 (u=(-5, -2), v=(3, -5), ab-gcd=1)
q:31, s:18, t:9 (fired at 7), i:-3, j:1, a:18, b:1 (u=(-5, -2), v=(3, -5), ab-gcd=1)
q:31, s:18, t:13 (fired at 11), i:-3, j:5, a:30, b:-19 (u=(-5, -2), v=(3, -5), ab-gcd=1)
q:31, s:18, t:16 (fired at 14), i:-2, j:1, a:13, b:-1 (u=(-5, -2), v=(3, -5), ab-gcd=1)
q:31, s:18, t:20 (fired at 18), i:-2, j:5, a:25, b:-21 (u=(-5, -2), v=(3, -5), ab-gcd=1)
q:31, s:18, t:24 (fired at 22), i:-1, j:2, a:11, b:-8 (u=(-5, -2), v=(3, -5), ab-gcd=1)
q:31, s:18, t:27 (fired at 25), i:-1, j:5, a:20, b:-23 (u=(-5, -2), v=(3, -5), ab-gcd=1)
q:31, s:18, t:37 (fired at 35), i:1, j:1, a:-2, b:-7 (u=(-5, -2), v=(3, -5), ab-gcd=1)
q:31, s:18, t:40 (fired at 38), i:1, j:4, a:7, b:-22 (u=(-5, -2), v=(3, -5), ab-gcd=1)
q:31, s:18, t:41 (fired at 39), i:1, j:5, a:10, b:-27 (u=(-5, -2), v=(3, -5), ab-gcd=1)
q:31, s:18, t:48 (fired at 46), i:2, j:5, a:5, b:-29 (u=(-5, -2), v=(3, -5), ab-gcd=1)
q:31, s:18, t:55 (fired at 53), i:3,

In [111]:
print(siever.prime_ideals_siever.p)
print(siever.prime_siever.p)

[2 7]
[ 2  3  5  7 11 13]


In [125]:
print(2*pi(2**15)+3)

7027
